In [2]:
from __future__ import print_function
import sys
import requests as rs
from bs4 import BeautifulSoup as bs
from datetime import date,datetime
import json
# from splinter import Browser
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re

## 原始版

In [ ]:
# f= open('e:/jupyter/TRAVEL/tripadvisor_1_14_test.txt','w')
home_url = 'https://www.tripadvisor.com.tw'
base_url='https://www.tripadvisor.com.tw/Attractions-g293913-Activities-oa{}-Taipei.html#ATTRACTION_LIST'
textNo=0
#景點分頁下載
for page in xrange(0,14,+1):
    title=(base_url.format(page*30))
#     print(title)
    res=rs.get(title)
    soup=bs(res.text,'html5lib')
    articles = soup.select('.photo_booking.non_generic .photo_link')
#分頁網站連接下載
    for article in articles:
        try:
            url= home_url + article['href']
        except IndexError:
            url = 'null'
#         print(url)
        res=rs.get(url)
        soup=bs(res.text,'html5lib')
        inner_url = url.replace('Reviews-','Reviews-or{}-')
        content = inner_url.split('-')[3].split('-')[0]

#             print(content)
#             if content != 'Reviews':
#                 print(inner_url = 'null')

        total_page =int(soup.select('.unified.pagination .pageNum.taLnk')[-1]['data-page-number'])
        print(type(total_page))
        for page in xrange(0,total_page-1):
            inner_title = (inner_url.format(page*10))    
#             print(inner_title)    
            inner_page_res = rs.get(inner_title)
            inner_page_bs = bs(inner_page_res.text ,'html5lib')
            total_userId = len(inner_page_bs.select('div.memberBadging.g10n'))

            for num in range(0,total_userId,+1):
            #使用者ID
                try:
                    UID = inner_page_bs.select('div.memberBadging.g10n')[num].select('div')[0]['id']\
                        .split('-')[0]\
                        .replace('UID_','')



        #     #================================================================================================ 
         #各使用者簡介

                    user_url = 'https://www.tripadvisor.com.tw/MemberOverlay?Mode=owa&uid={}&c=&src=&fus=false&partner=false&LsoId=&metaReferer=Attraction_Review'
                    user_infopage = (user_url.format(UID))
        #                        print(user_infopage)
                    user_res = rs.get(user_infopage)
                    user_soup = bs(user_res.text,'html5lib')
    #                                 print(user_soup.select('div.memberreviewbadge')[0].select('div.badgeinfo')[0].text)
                    def user_function (a,b):
                        return user_soup.select(a)[0].select(b)[0].text
                  #使用者簡介\
#                     try:
                    user_name = user_function('.memberOverlayRedesign.g10n','.username')
                    user_time = user_function('.memberOverlayRedesign.g10n','.memberdescription')
                    user_counts = user_function('.memberOverlayRedesign.g10n','.counts') 
                    user_homepage = home_url + user_soup.select('div.baseNav')[0].select('a')[0]['href']
                    user_comments = user_function('.lowerMemberOverlay.wrap','.reviewchart.wrap.container')
#                     print(user_comments+user_counts,'\n')#使用者評論數
#                         print(user_name)#使用者名字
#                         print(user_time)#只用者註冊時間
#                         print(user_homepage)#只用者個人頁面網址
#===============================================================================================================
                    #使用者評論截取
                    broswer = webdriver.Firefox()
                    broswer.get(user_homepage)
                    broswer.maximize_window()
                    # broswer.execute_script('window.scrollTo(0,document.body.scrollHeight);')
                    soup = bs(broswer.page_source)
                    article = soup.select('li.cs-review')
                    page_num = len(soup.select('button.cs-paginate-goto'))
                    if page_num > 2:
                        while soup.select('button#cs-paginate-next')[0]['class'][0] != 'disabled':
                            for i in article:
                                title = i.select('.cs-review-location')[0].text
                                target = i.select('.cs-review-title')[0].text
                                url = home_url+i.select('.cs-review-location a')[0]['href']
                                rating = i.select('.cs-review-rating')[0].select('div.rate img')[0]['content']
                        #         time.sleep(1) 
#                                 print(title + target)
#                                 print(url)
#                                 print('評等'+rating)
                            broswer.execute_script('window.scrollTo(0,document.body.scrollHeight);')
                            #         broswer.find_element_by_xpath("//div[@id='BODYCON']/div[3]/div[4]/div/button[x+2]").click()
                            broswer.find_element_by_id("cs-paginate-next").click()
                            soup = bs(broswer.page_source)
                            article = soup.select('li.cs-review')
                            for i in article:    
                                title = i.select('.cs-review-location')[0].text
                                title_date = i.select('.cs-review-date')[0].text
                                target = i.select('.cs-review-title')[0].text
                                url = home_url+i.select('.cs-review-location a')[0]['href']
                                rating = i.select('.cs-review-rating')[0].select('div.rate img')[0]['content']
                                #使用者發文內容
                                user_res2 = rs.get(url)
                                user_soup2 = bs(user_res2.text,'html5lib')
                                url2 = home_url+user_soup2.select('div.quote a')[0]['href']
                                #使用者發文文章的連結
                                user_res3 = rs.get(url2) 
                                user_soup3 = bs(user_res3.text,'html5lib')
                                inner_note = user_soup3.select('div.entry')[0].text
                                broswer.close()  
                        #     broswer.implicitly_wait(3)
#                                 print(title + target)
#                                 print(title_date)
#                                 print(url)
#                                 print('評等'+rating)
#                                 print(inner_note,'-----------------------------------------')


                    else:
                        soup = bs(broswer.page_source)
                        article = soup.select('li.cs-review')
                        for i in article:
                            title = i.select('.cs-review-location')[0].text
                            title_date = i.select('.cs-review-date')[0].text
                            target = i.select('.cs-review-title')[0].text
                            url = home_url+i.select('.cs-review-location a')[0]['href']
                            rating = i.select('.cs-review-rating')[0].select('div.rate img')[0]['content']
                            #使用者發文內容
                            user_res2 = rs.get(url)
                            user_soup2 = bs(user_res2.text,'html5lib')
                            url2 = home_url+user_soup2.select('div.quote a')[0]['href']
                            #使用者發文文章的連結
                            user_res3 = rs.get(url2) 
                            user_soup3 = bs(user_res3.text,'html5lib')
                            inner_note = user_soup3.select('div.entry')[0].text


                    #         time.sleep(1) 
#                             print(title + target)
#                             print(title_date)
#                             print(url)
#                             print('評等'+rating)
#                             print(inner_note,'-----------------------------------------')

                    broswer.close()            
                except IndexError:
                    user_comments = 'null'
#                         print(user_homepage)


                finally:        

#                     print(user_name)#使用者名字
#                     print(user_time)#只用者註冊時間
#                     print(user_homepage)#只用者個人頁面網址

                #使用者個人頁面資訊
                    try:
                        user_res2 = rs.get(user_homepage)
                        user_soup2 = bs(user_res2.text,'html5lib')
                        print("4"+user_name_info)
                        user_point = user_soup2.select('div.points_info.tripcollectiveinfo')[0].text
                        print("5"+user_point)
                        user_lv = user_soup2.select('div.level.tripcollectiveinfo')[0].text
                        print("6"+user_lv)
                        user_badge = user_soup2.select('div.badgeList.badgeListLoggedOut')[0].text
                        print("7"+user_badge)
                        div = []
                        for y in range(0,target_num):
                            user_target = user_soup2.select('div.tagBubble.unclickable')[y].text+','
#                             div.append(user_target.decode('unicode_escape').encode('utf-8'))
                        # broswer.implicitly_wait(3)

#     broswer.find_element_by_css_selector("button.cs-paginate-goto.active").click()
# time.sleep(2)    



                        #使用者評論等級                        
                            print((user_target).strip(','))#使用者標籤
#                             print(user_name_info+user_point,'\n')#使用者獲得點數
#                             print(user_badge,'\n')#使用者獲得標章

                    except:
                       user_target = 'null'


                    finally:   
                        print(user_name_info+user_point,'\n')#使用者獲得點數
                        print(user_badge,'\n')#使用者獲得標章


                        print('==========================================================')



#                         with open('E:/project/test-folder/{}.txt'.format(str(textNo).zfill(8)),'w') as f:
#                             f.write(user_name.encode("UTF-8")+"\n"\
#                                     +user_time.encode("UTF-8")+" "+user_target.encode('UTF-8')+"\n"\
#                                     +user_name_info.encode('UTF-8')+" "+user_point.encode('UTF-8')+"\n"\
#                                     +user_badge.encode("UTF-8")+"\n"\
#                                     +url.encode("UTF-8")+"\n"\
#                                     +user_comments.encode("UTF-8")+" "+user_counts.encode("UTF-8")+"\n"\
#                                     +title_date.encode("UTF-8")+"\n"\
#                                     +inner_note.encode("UTF-8")
#                                    )
#                         textNo += 1    

# time.sleep(5)

# f.close()


## 評論人重複值還沒有解決~~

In [ ]:
home_url = 'https://www.tripadvisor.com.tw'
base_url='https://www.tripadvisor.com.tw/Attractions-g293913-Activities-oa{}-Taipei.html#ATTRACTION_LIST'
textNo=0
#景點分頁下載
for page in xrange(0,14,+1):
    title=(base_url.format(page*30))
#     print(title)
    res=rs.get(title)
    soup=bs(res.text,'html5lib')
    for n in soup.select('.photo_booking.non_generic .photo_link'):    
    #分頁網站連接下載
        try:
            url='https://www.tripadvisor.com.tw'+n['href']
            if(url.split('_')[1].split('-')[0]=='Review'):
                print(url)      
            else:
                url='null'
                print("怪怪的")
            #============分頁挑選========#    
            res=rs.get(url)
            soup=bs(res.text,'html5lib')        
            inner_url = url.replace('Reviews-','Reviews-or{}-')
#             print("1"+ inner_url)
            #==========各景點評論的初始頁==================#
            content = inner_url.split('-')[3].split('-')[0]
            total_page =int(soup.select('.unified.pagination .pageNum.taLnk')[-1]['data-page-number'])
            print(total_page)
            #=====各評論的總頁數===================#
            for page in xrange(0,total_page-1):
                inner_title = (inner_url.format(page*10))    
#                 print("2"+ inner_title)
                #===========各評論的評論內容========#    
                inner_page_res = rs.get(inner_title)
                inner_page_bs = bs(inner_page_res.text ,'html5lib')
                total_userId = len(inner_page_bs.select('div.memberBadging.g10n'))
                for num in range(0,total_userId,+1):               
                    try:
                       
                        UID = inner_page_bs.select('div.memberBadging.g10n')[num].select('div')[0]['id']\
                            .split('-')[0]\
                            .replace('UID_','')
                            #使用者ID#
                        member_id = UID  
                        print(UID)
                       
                        user_url = 'https://www.tripadvisor.com.tw/MemberOverlay?Mode=owa&uid={}&c=&src=&fus=false&partner=false&LsoId=&metaReferer=Attraction_Review'
                        user_infopage = (user_url.format(UID))
                        print(user_infopage)
                        print("++++++++++++++++++++++++++")
                        #各使用者簡介#
                        user_res = rs.get(user_infopage)
                        user_soup = bs(user_res.text,'html5lib')
#                                         print(user_soup.select('div.memberreviewbadge')[0].select('div.badgeinfo')[0].text)
                        def user_function (a,b):
                            return user_soup.select(a)[0].select(b)[0].text                
                        user_name = user_function('.memberOverlayRedesign.g10n','.username')                        
#                         print(user_name)                        
                        user_time = user_function('.memberOverlayRedesign.g10n','.memberdescription')
#                         print(type(user_time))
#                         print("1"+user_time)
                        user_counts = user_function('.memberOverlayRedesign.g10n','.counts')
                        member_remark=user_counts
#                         print("2"+member_remark)
                        user_homepage = home_url + user_soup.select('div.baseNav')[0].select('a')[0]['href']
                        print(user_homepage)
                        #使用者個人頁面網址.可以用來做字典剔除
#===============================================================================================================# 
                        #使用者評論截取
#                         broswer = webdriver.Firefox()
#                         broswer.get(user_homepage)
#                         broswer.maximize_window()
#                         # broswer.execute_script('window.scrollTo(0,document.body.scrollHeight);')
#                         soup = bs(broswer.page_source)
#                         article = soup.select('li.cs-review')
#                         page_num = len(soup.select('button.cs-paginate-goto'))
                        #==========================================#
                        print("************************")
                        user_res2 = rs.get(user_homepage)
                        print(user_res2)
                        user_soup2 = bs(user_res2.text,'html5lib')
                        user_gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text
                        print("3"+user_gender)
                        user_country=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.hometown')[0].text
                        print("3"+user_country)
                        user_name_info = user_soup2.select('div.modules-membercenter-progress-header')[0].select('div.name')[0].text
                        print("4"+user_name_info)
                        user_point = user_soup2.select('div.points_info.tripcollectiveinfo')[0].text
                        print("5"+user_point)
                        user_lv = user_soup2.select('div.level.tripcollectiveinfo')[0].text
                        print("6"+user_lv)
                        user_badge = user_soup2.select('div.badgeList.badgeListLoggedOut')[0].text
                        print("7"+user_badge)
#                         target_num = len(user_soup2.select('div.tagBubble.unclickable'))
#                         for y in range(0,target_num):
#                             user_target = user_soup2.select('div.tagBubble.unclickable')[y].text+','
#                         #使用者評論等級                        
#                             print((user_target).strip(','))#使用者標籤
#                             print(user_name_info+user_point,'\n')#使用者獲得點數
#                             print(user_badge,'\n')#使用者獲得標章
                        
                                
#                             print(user_name_info+user_point,'\n')#使用者獲得點數
#                             print(user_badge,'\n')#使用者獲得標章 
    # #                     print(user_name)#使用者名字
    # #                     print(user_time)#只用者註冊時間
    # #                     print(user_homepage)#只用者個人頁面網址

#                         broswer.close()            
                    except:
                        user_target = 'null'
                        user_badge='null' 
                        user_lv='null' 
#                     finally:
#                         print("ok1")
        except IndexError:
                print("=====================")
                print("出現Skip")


In [ ]:
    #===============================================================================================================# 
    #                     #使用者評論截取
    #                     broswer = webdriver.Firefox()
    #                     broswer.get(user_homepage)
    #                     broswer.maximize_window()
    #                     # broswer.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    #                     soup = bs(broswer.page_source)
    #                     article = soup.select('li.cs-review')
    #                     page_num = len(soup.select('button.cs-paginate-goto'))
    #                     if page_num > 2:
    #                         while soup.select('button#cs-paginate-next')[0]['class'][0] != 'disabled':
    #                             for i in article:
    #                                 title = i.select('.cs-review-location')[0].text
    #                                 target = i.select('.cs-review-title')[0].text
    #                                 url = home_url+i.select('.cs-review-location a')[0]['href']
    #                                 rating = i.select('.cs-review-rating')[0].select('div.rate img')[0]['content']
    #                         #         time.sleep(1) 
    # #                                 print(title + target)
    # #                                 print(url)
    # #                                 print('評等'+rating)
    #                             broswer.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    #                             #         broswer.find_element_by_xpath("//div[@id='BODYCON']/div[3]/div[4]/div/button[x+2]").click()
    #                             broswer.find_element_by_id("cs-paginate-next").click()
    #                             soup = bs(broswer.page_source)
    #                             article = soup.select('li.cs-review')
    #                             for i in article:    
    #                                 title = i.select('.cs-review-location')[0].text
    #                                 title_date = i.select('.cs-review-date')[0].text
    #                                 target = i.select('.cs-review-title')[0].text
    #                                 url = home_url+i.select('.cs-review-location a')[0]['href']
    #                                 rating = i.select('.cs-review-rating')[0].select('div.rate img')[0]['content']
    #                                 #使用者發文內容
    #                                 user_res2 = rs.get(url)
    #                                 user_soup2 = bs(user_res2.text,'html5lib')
    #                                 url2 = home_url+user_soup2.select('div.quote a')[0]['href']
    #                                 #使用者發文文章的連結
    #                                 user_res3 = rs.get(url2) 
    #                                 user_soup3 = bs(user_res3.text,'html5lib')
    #                                 inner_note = user_soup3.select('div.entry')[0].text
    # #                                 broswer.close()  
    #                         #     broswer.implicitly_wait(3)
    # #                                 print(title + target)
    # #                                 print(title_date)
    # #                                 print(url)
    # #                                 print('評等'+rating)
    # #                                 print(inner_note,'-----------------------------------------')


    #                     else:
    #                         soup = bs(broswer.page_source)
    #                         article = soup.select('li.cs-review')
    #                         for i in article:
    #                             title = i.select('.cs-review-location')[0].text
    #                             title_date = i.select('.cs-review-date')[0].text
    #                             target = i.select('.cs-review-title')[0].text
    #                             url = home_url+i.select('.cs-review-location a')[0]['href']
    #                             rating = i.select('.cs-review-rating')[0].select('div.rate img')[0]['content']
    #                             #使用者發文內容
    #                             user_res2 = rs.get(url)
    #                             user_soup2 = bs(user_res2.text,'html5lib')
    #                             url2 = home_url+user_soup2.select('div.quote a')[0]['href']
    #                             #使用者發文文章的連結
    #                             user_res3 = rs.get(url2) 
    #                             user_soup3 = bs(user_res3.text,'html5lib')
    #                             inner_note = user_soup3.select('div.entry')[0].text


    #                     #         time.sleep(1) 
    # #                             print(title + target)
    # #                             print(title_date)
    # #                             print(url)
    # #                             print('評等'+rating)
    # #                             print(inner_note,'-----------------------------------------')

    #                     broswer.close()
                    except:
                     except IndexError:                        
                        user_comments = 'null'
                        print(user_homepage)


    #                 finally:        
                                                    #使用者個人頁面資訊
                    try:
                        user_res2 = rs.get(user_homepage)
                        user_soup2 = bs(user_res2.text,'html5lib')
                        user_gender=user_soup2.select('modules-membercenter-member-profile ')[0].select('p')[0].text
                        print("3"+user_gender)
                        user_name_info = user_soup2.select('div.modules-membercenter-progress-header')[0].select('div.name')[0].text
                        print("4"+user_name_info)
                        user_point = user_soup2.select('div.points_info.tripcollectiveinfo')[0].text
                        print("5"+user_point)
                        user_lv = user_soup2.select('div.level.tripcollectiveinfo')[0].text
                        print("6"+user_lv)
                        user_badge = user_soup2.select('div.badgeList.badgeListLoggedOut')[0].text
                        print("7"+user_badge)
                        target_num = len(user_soup2.select('div.tagBubble.unclickable'))
                        div = []
                        for y in range(0,target_num):
                            user_target = user_soup2.select('div.tagBubble.unclickable')[y].text+','
                        #使用者評論等級                        
                            print((user_target).strip(','))#使用者標籤
#                             print(user_name_info+user_point,'\n')#使用者獲得點數
#                             print(user_badge,'\n')#使用者獲得標章
                        
                                
#                             print(user_name_info+user_point,'\n')#使用者獲得點數
#                             print(user_badge,'\n')#使用者獲得標章 
    # #                     print(user_name)#使用者名字
    # #                     print(user_time)#只用者註冊時間
    # #                     print(user_homepage)#只用者個人頁面網址

    #                


    #                         print('==========================================================')



    #                         with open('E:/project/test-folder/{}.txt'.format(str(textNo).zfill(8)),'w') as f:
    #                             f.write(user_name.encode("UTF-8")+"\n"\
    #                                     +user_time.encode("UTF-8")+" "+user_target.encode('UTF-8')+"\n"\
    #                                     +user_name_info.encode('UTF-8')+" "+user_point.encode('UTF-8')+"\n"\
    #                                     +user_badge.encode("UTF-8")+"\n"\
    #                                     +url.encode("UTF-8")+"\n"\
    #                                     +user_comments.encode("UTF-8")+" "+user_counts.encode("UTF-8")+"\n"\
    #                                     +title_date.encode("UTF-8")+"\n"\
    #                                     +inner_note.encode("UTF-8")
    #                                    )
    #                         textNo += 1    

    # time.sleep(5)

    # f.close()

In [ ]:
text = '''Amazon has long favored growth over profits, and in its nearly 22-year lifespan, the company has poured resources into building out a sprawling logistics infrastructure dedicated to giving you what you want almost immediately. Why should you have to wait for days (or, God forbid, leave your house) when you need toilet paper?

Stephenie Landry is the force responsible for the instantaneousness of your gratification—as vice president of Amazon Prime Now, she conceptualized how the company would bring one-hour deliveries to life and assembled the team to work on the problem. “We have something at Amazon called the working-backward process,” Landry says. “We write a press release saying what we would announce to the world, and when I was writing the product concept, I wrote that the experience of Prime Now would be ‘magical.’” Since launching in December 2014, the service has gone live in four countries and more than 30 metropolitan areas around the world. And, as rumors swirl that Amazon may also be working on a global delivery network, Prime Now increasingly looks to be a scaled-down version of the company’s long-term play: to be in complete control of the flow of products in its supply chain, from factories in China and India all the way to your doorstep. That grand plan would involve not only trucks, cargo planes, and drones but also hundreds of thousands of humans working in warehouses and as couriers. And if anything like that aspirational picture of Prime Now actually emerges, UPS and FedEx should probably start prepping their contingency plans as soon as, well, now. —Davey Alba
'''
words = text.split(' ')

dic = {}
for w in words:
    if w not in dic:
        dic[w] = 1
    else:
        dic[w] += 1
print(dic)

## 評論人重複值還沒有解決~~準備解決版本

In [ ]:
home_url = 'https://www.tripadvisor.com.tw'
base_url='https://www.tripadvisor.com.tw/Attractions-g293913-Activities-oa{}-Taipei.html#ATTRACTION_LIST'
textNo=0
#景點分頁下載
for page in xrange(0,14,+1):
    title=(base_url.format(page*30))
#     print(title)
    res=rs.get(title)
    soup=bs(res.text,'html5lib')
    for n in soup.select('.photo_booking.non_generic .photo_link'):    
    #分頁網站連接下載
        try:
            url='https://www.tripadvisor.com.tw'+n['href']
            if(url.split('_')[1].split('-')[0]=='Review'):
                print(url)      
            else:
                url='null'
                print("怪怪的")
            #============分業挑選========#    
            res=rs.get(url)
            soup=bs(res.text,'html5lib')        
            inner_url = url.replace('Reviews-','Reviews-or{}-')
#             print("1"+ inner_url)
            #==========各景點評論的初始頁==================#
            content = inner_url.split('-')[3].split('-')[0]
            total_page =int(soup.select('.unified.pagination .pageNum.taLnk')[-1]['data-page-number'])
            print(total_page)
            #=====各評論的總頁數===================#
            for page in xrange(0,total_page-1):
                inner_title = (inner_url.format(page*10))    
#                 print("2"+ inner_title)
                #===========各評論的評論內容========#    
                inner_page_res = rs.get(inner_title)
                inner_page_bs = bs(inner_page_res.text ,'html5lib')
                total_userId = len(inner_page_bs.select('div.memberBadging.g10n'))
                for num in range(0,total_userId,+1):               
                    try:
                        dic = {}
                        UID = inner_page_bs.select('div.memberBadging.g10n')[num].select('div')[0]['id']\
                            .split('-')[0]\
                            .replace('UID_','')
                            #使用者ID#
                        UID= UID.split(' ')
                        for UID_number in UID:
                            if UID_number not in dic:
                                dic[UID_number] = 1                               
                            else:
                                print("UID_number 重複值")
                                dic[UID_number] += 1
#                         print(UID)
                        print(dic)
                    except IndexError:
                        print("==============")
                        print("UID空值")

        except IndexError:
                print("=====================")
                print("出現Skip")


In [ ]:
user_homepage='https://www.tripadvisor.com.tw/members/doris7116'
# https://www.tripadvisor.com.tw/MemberProfile-a_uid.3806B087E1EEEDBA38F7E690310B5144
#     'https://www.tripadvisor.com.tw/members/doris7116'
user_res2 = rs.get(user_homepage)
print(user_res2)
user_soup2 = bs(user_res2.text,'html5lib')
user_year=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text
match=re.search(r'\d{2}-\d{2}', user_year)
if match:
    member_year=match.group(0)
    print(member_year)
else:
    print("not found")
    member_year='null'
# print("3"+user_year)    
user_gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8')
# print("3-1"+user_gender) 
gender=['預設值','男性','女性']
for i in range(0,3):
    match=re.search(gender[i],user_gender)
    if match:        
        user_gender=user_gender.replace(gender[i],str(i))
#         print(user_gender)
        match = re.search(r'\d', user_gender)
        gender_id=match.group(0)
        print(gender_id)
    else:
#         print("not found")
         gender_id='null'  
user_country=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.hometown')[0].text
user_country='台灣台中市'
country=['0','台灣','香港','馬來西亞','新加坡','美國','加拿大','澳洲','紐西蘭','英國','中國','韓國','日本','印尼','泰國','法國','德國','俄羅斯','哈薩克','葡萄牙','巴西','西班牙','阿根廷','秘魯','荷蘭','義大利']
for i in range(0,25):
    match=re.search(country[i],user_country)
    if match:
        country_id=user_country.replace(country[i],str(i))
#         print(country_id)
        match = re.search(r'\d+', country_id)
        country_id=match.group(0)
        print(country_id)
    else:
#         print("not found")
         country_id='null'
# print("4"+user_country)
user_name_info = user_soup2.select('div.modules-membercenter-progress-header')[0].select('div.name')[0].text
# print("4"+user_name_info)
# user_point = user_soup2.select('div.points_info.tripcollectiveinfo')[0].text
# print("5"+user_point)
user_lv = user_soup2.select('div.level.tripcollectiveinfo')[0].text
print("6"+user_lv)
match=re.search(r'\d',user_lv)
if match:
    user_lv=match.group(0)
    print(user_lv)
else:
    user_lv='null'
    
num = len(user_soup2.select('li.content-info'))
for i in range(0,num):
    comm_times = user_soup2.select('li.content-info')[i].select('a')[0]    
#     print(r['name'].decode('utf-8'))
    if comm_times['name'].decode('utf-8') == 'reviews': 
#         print(comm_times.text)
        match=re.search(r'\d*',comm_times.text)
        if match:            
            comm_times=match.group(0)
            print(comm_times)
            comm_times=comm_times            
#         else:
#         comm_times='null'       
    else:        
        comm_times='null'
    comm_times = user_soup2.select('li.content-info')[i].select('a')[0]     
    if comm_times['name'].decode('utf-8') == 'ratings':
#         print(comm_times.text)
        match=re.search(r'\d*',comm_times.text)
        if match:            
            comm_raking=match.group(0)
            print(comm_raking)
            comm_raking=comm_raking            
#         else:
#         comm_times='null'       
    else:        
        comm_raking='null'   

# print("8"+user_cont_info)
# print(len(user_cont_info))
# match=re.search(r'\d+'+'評論',user_cont_info)
# user_cont_info=match.group(0)
# print(user_cont_info)


In [4]:
phones = ['我的電話是 0912345678 Call Me', '0922-222-222 我餓了', \
          '等妳呦 0911-222333', '真心不騙 09123482384060192381']

for phone in phones:
    m = re.search(r'09\d+', phone)
    print(m.group(0))
print("=====================")
for phone in phones:
    m = re.search(r'09\d{2}-?\d{3}-?\d{3}', phone) # r'09\d\d'
    print(m.group(0))

0912345678
0922
0911
09123482384060192381
0912345678
0922-222-222
0911-222333
0912348238


In [3]:
from __future__ import print_function
import sys
import requests as rs
from bs4 import BeautifulSoup as bs
from datetime import date,datetime
import json
# from splinter import Browser
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
user_homepage='https://www.tripadvisor.com.tw/members/singaDaave'
user_res2 = rs.get(user_homepage)
print(user_res2)
user_soup2 = bs(user_res2.text,'html5lib')
user_country=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.hometown')[0].text.encode('utf-8')


country=['0','台灣','香港','馬來西亞','新加坡','美國','加拿大','澳洲','紐西蘭','英國','中國','韓國','日本','印尼','泰國','法國','德國','俄羅斯','哈薩克','葡萄牙','巴西','西班牙','阿根廷','秘魯','荷蘭','義大利']
for i in range(0,25):
    match=re.search(country[i],user_country)
    if match:
        country_id=user_country.replace(country[i],str(i))
        print(country_id)
        match = re.search(r'\d+', country_id)
        country_id=match.group(0)
        print(country_id)
    else:
        print("not found")
        country_id='null'
        
# 1	台灣
# 2	香港
# 3	馬來西亞
# 4	新加坡
# 5	美國
# 6	加拿大
# 7	澳洲
# 8	紐西蘭
# 9	英國
# 10	中國
# 11	韓國
# 12	日本
# 13	印尼
# 14	泰國
# 15	法國
# 16	德國
# 17	俄羅斯
# 18	哈薩克
# 19	葡萄牙
# 20	巴西
# 21	西班牙
# 22	阿根廷
# 23	秘魯
# 24	荷蘭
# 25	義大利

<Response [200]>
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
not found


In [ ]:
import re
user_target='饕客 背包客 家庭渡假旅客 安詳與寧靜追尋者 尋求經濟實惠旅行的旅客'
print(type(user_target))
style=['0','注重豪華旅遊的旅客','家庭渡假旅客','60歲以上旅客','海灘愛好者','都市探索者','素食主義者','尋求刺激的旅客','自然愛好者','安詳與寧靜追尋者','饕客','夜生活愛好者','儉約的旅客','引領流行的旅客','熱衷購物的旅客','熟悉當地環境的旅客','尋求經濟實惠旅行的旅客','背包客','藝術與建築愛好者','歷史愛好者']
for i in range(0,19):
    match=re.search(style[i],user_target)
    if match:        
        user_target=user_target.replace(style[i],str(i))    
print(user_target)
# 1. luxury : 注重豪華旅遊的旅客
# 2. family_travel: 家庭渡假旅客
# 3.  over_60yrs : 60 歲以上旅客
# 4. beach_lover : 海灘愛好者
# 5. city_lover: 都市探索者
# 6. vegetarian: 素食主義者
# 7. excited_lover : 尋求刺激的旅客
# 8. natural_lover : 自然愛好者
# 9. peace_lover : 安詳與寧靜追尋者
# 10. food_lover : 饕客
# 11. nightlife_lover : 夜生活愛好者
# 12. frugal_traveler : 儉約的旅客
# 13. fashion_leader : 引領流行的旅客
# 14. shopping_lover: 熱衷購物的旅客
# 15. localguide_traveler : 熟悉當地環境的旅客
# 16. highCP_persuer : 尋求經濟實惠旅行的旅客
# 17. backpackers: 背包客
# 18. arts_lover : 藝術與建築愛好者
# 19. history_lover : 歷史愛好者
 

In [ ]:
user_homepage='https://www.tripadvisor.com.tw/members/doris7116'
user_res2 = rs.get(user_homepage)
print(user_res2)
user_soup2 = bs(user_res2.text,'html5lib')
user_year=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text
match=re.search(r'\d{2}-\d{2}', user_year)
if match:
    member_year=match.group(0)
    print(member_year)
else:
    print("not found")
    member_year='null'
# print(user_year)
# print(type(user_year))

In [ ]:
# user_homepage='https://www.tripadvisor.com.tw/members/doris7116'
# user_res2 = rs.get(user_homepage)
# print(user_res2)
# user_soup2 = bs(user_res2.text,'html5lib')
# user_year=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text
# match=re.search(r'\d{2}-\d{2}', user_year)
# if match:
#     member_year=match.group(0)
#     print(member_year)
# else:
#     print("not found")
#     member_year='null'

# try:
#                             gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8')                        
#                             print(gender)
#                             genders=['預設值','男性','女性','男','女']
#                             for i in range(0,5):
#                                 match=re.search(genders[i],gender)                                                               
#                                 if match:
#                                     print(match.group(0))
#                                     gender=match.group(0)
#                                     print(gender)
# #                                     gender=match.group(0)
# #                                     print(gender)
# #                                     gender=gender
# #                                     print("***********")
# #                                     print(gender)
# #                                     print(type(gender))
#                                 else: 
#                                     gender='null'
#     #                                     print("not found")
#                         except:
#                             gender='null'            
user_gender='註冊時間：2014年11月 25-34 歲 男性  '
print(user_gender)
s=len(user_gender.split())
print(s)
genders=['預設值','男性','女性','男','女']
for i in range(0,4):    
    match=re.search(gender[i],user_gender)
    if match:
        user_gender=user_gender.replace(gender[i],str(i)) 
        print(user_gender)
    else: 
        print("not found")
#         user_gender='null'
    
        print("3"+user_gender)





In [ ]:
import re
user_homepage='https://www.tripadvisor.com.tw/members/%E8%93%8B%E7%91%9E%E7%89%B9'
# https://www.tripadvisor.com.tw/members/%E8%93%8B%E7%91%9E%E7%89%B9
user_res2 = rs.get(user_homepage)
print(user_res2)
user_soup2 = bs(user_res2.text,'html5lib')
user_year=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text
match=re.search(r'\d{2}-\d{2}', user_year)
if match:
    member_year=match.group(0)
    print(member_year)
else:
    print("not found")
    member_year='null'
print("3"+user_year)    
user_country=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8')
print(user_country)
country=['預設值','男性','女性']
for i in range(0,3):
    match=re.search(country[i],user_country)
    if match:
        print(country[i])
#         user_country=user_country.replace(country[i],str(i))
#         print(user_country)
#         match = re.search(r'\d', user_country)
#         print(match.group(0))
#         country_id=match.group(0)
#         print(country_id)
     else:
        print("not found")
        country_id='null'

# user_target='饕客 背包客 家庭渡假旅客 安詳與寧靜追尋者 尋求經濟實惠旅行的旅客'
# print(type(user_target))
# style=['0','注重豪華旅遊的旅客','家庭渡假旅客','60歲以上旅客','海灘愛好者','都市探索者','素食主義者','尋求刺激的旅客','自然愛好者','安詳與寧靜追尋者','饕客','夜生活愛好者','儉約的旅客','引領流行的旅客','熱衷購物的旅客','熟悉當地環境的旅客','尋求經濟實惠旅行的旅客','背包客','藝術與建築愛好者','歷史愛好者']
# for i in range(0,19):
#     match=re.search(style[i],user_target)
#     if match:        
#         user_target=user_target.replace(style[i],str(i))    
# print(user_target)

In [ ]:
url = ('https://www.tripadvisor.com.tw/members/tsai95862016')
res = rs.get(url)
soup = bs(res.text,'html5lib')
num = len(soup.select('li.content-info'))
for i in range(0,num):

    r = soup.select('li.content-info')[i].select('a')[0]
    
#     print(r['name'].decode('utf-8'))
    if r['name'].decode('utf-8') == 'reviews':
        print(r.text)
    elif r['name'].decode('utf-8') == 'ratings':
        print(r.text)

In [ ]:
url = ('https://www.tripadvisor.com.tw/members/tsai95862016')
res = rs.get(url)
soup = bs(res.text,'html5lib')
num = len(soup.select('li.content-info'))
for i in range(0,num):
    comm_times = soup.select('li.content-info')[i].select('a')[0]    
#     print(r['name'].decode('utf-8'))
    if comm_times['name'].decode('utf-8') == 'reviews': 
#         print(comm_times.text)
        match=re.search(r'\d...',comm_times.text)
        if match:            
            comm_times=match.group(0)
            print(comm_times)
            comm_times=comm_times            
#         else:
#         comm_times='null'       
    else:        
        comm_times='null'
    comm_times = soup.select('li.content-info')[i].select('a')[0]     
    if comm_times['name'].decode('utf-8') == 'ratings':
#         print(comm_times.text)
        match=re.search(r'\d...',comm_times.text)
        if match:            
            comm_raking=match.group(0)
            print(comm_raking)
            comm_raking=comm_raking            
#         else:
#         comm_times='null'       
    else:        
        comm_raking='null'   



In [ ]:
from __future__ import print_function
import sys
import requests as rs
from bs4 import BeautifulSoup as bs
from datetime import date,datetime
import json
# from splinter import Browser
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
home_url = 'https://www.tripadvisor.com.tw'
base_url='https://www.tripadvisor.com.tw/Attractions-g293913-Activities-oa{}-Taipei.html#ATTRACTION_LIST'
textNo=0
#景點分頁下載
for page in xrange(0,14,+1):
    title=(base_url.format(page*30))
#     print(title)
    res=rs.get(title)
    soup=bs(res.text,'html5lib')
    for n in soup.select('.photo_booking.non_generic .photo_link'):    
    #分頁網站連接下載
        try:
            url='https://www.tripadvisor.com.tw'+n['href']
            if(url.split('_')[1].split('-')[0]=='Review'):
                print(url)      
            else:
                url='null'
                print("怪怪的")
            #============分頁挑選========#    
            res=rs.get(url)
            soup=bs(res.text,'html5lib')        
            inner_url = url.replace('Reviews-','Reviews-or{}-')
#             print("1"+ inner_url)
            #==========各景點評論的初始頁==================#
            content = inner_url.split('-')[3].split('-')[0]
            total_page =int(soup.select('.unified.pagination .pageNum.taLnk')[-1]['data-page-number'])
            print(total_page)
            #=====各評論的總頁數===================#
            for page in xrange(0,total_page-1):
                inner_title = (inner_url.format(page*10))    
#                 print("2"+ inner_title)
                #===========各評論的評論內容========#    
                inner_page_res = rs.get(inner_title)
                inner_page_bs = bs(inner_page_res.text ,'html5lib')
                total_userId = len(inner_page_bs.select('div.memberBadging.g10n'))
                for num in range(0,total_userId,+1):               
                    try:
                        UID = inner_page_bs.select('div.memberBadging.g10n')[num].select('div')[0]['id']\
                            .split('-')[0]\
                            .replace('UID_','')
                            #使用者ID#
                        member_id = UID  
                        print(UID)                       
                        user_url = 'https://www.tripadvisor.com.tw/MemberOverlay?Mode=owa&uid={}&c=&src=&fus=false&partner=false&LsoId=&metaReferer=Attraction_Review'
                        user_infopage = (user_url.format(UID))
                        print(user_infopage)
                        print("++++++++++++++++++++++++++")
                        #各使用者簡介#
                        user_res = rs.get(user_infopage)
                        user_soup = bs(user_res.text,'html5lib')
#                                         print(user_soup.select('div.memberreviewbadge')[0].select('div.badgeinfo')[0].text)
                        def user_function (a,b):
                            return user_soup.select(a)[0].select(b)[0].text                
                        user_name = user_function('.memberOverlayRedesign.g10n','.username')                        
#                         print(user_name)                        
                        user_time = user_function('.memberOverlayRedesign.g10n','.memberdescription')
#                         print(type(user_time))
#                         print("1"+user_time)
                        user_counts = user_function('.memberOverlayRedesign.g10n','.counts')
                        member_remark=user_counts
#                         print("2"+member_remark)
                        user_homepage = home_url + user_soup.select('div.baseNav')[0].select('a')[0]['href']
                        print(user_homepage)
#                         user_homepage='https://www.tripadvisor.com.tw/members/doris7116'
                        # https://www.tripadvisor.com.tw/MemberProfile-a_uid.3806B087E1EEEDBA38F7E690310B5144
                        #     'https://www.tripadvisor.com.tw/members/doris7116'
                        user_res2 = rs.get(user_homepage)
                        print(user_res2)
                        user_soup2 = bs(user_res2.text,'html5lib')
                        user_year=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text
                        match=re.search(r'\d{2}-\d{2}', user_year)
                        if match:
                            print(user_year)
                            member_year=match.group(0)
                            print(member_year)
                        else:
                            print("not found")
                            member_year='null'
                        # print("3"+user_year)                        
                        try:
                            gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8')                            
                            print(gender)
#                             print("11111111")
                            s=len(user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8').split())
#                             print(s)                            
                            for y in range(0,s):               
                                gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8').split()[y]
#                                 print(gender)                            
                                s=['預設值','男','女']
                                for i in range(0,3):
                                    match=re.search(s[i],gender)
#                                     print("------------")                                
                                    if match:
                                        d=gender.replace(s[i],str(i))
                                        match=re.search(r'\d', d)
                                        gender=match.group(0)
                                        print(gender)
                   
                        except:
                            gender='null'
                        #======gender==========#    
                        user_country=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.hometown')[0].text.encode('utf-8')
                        print(user_country)
                        country=['0','台灣','香港','馬來西亞','新加坡','美國','加拿大','澳洲','紐西蘭','英國','中國','韓國','日本','印尼','泰國','法國','德國','俄羅斯','哈薩克','葡萄牙','巴西','西班牙','阿根廷','秘魯','荷蘭','義大利','南韓']
                        for i in range(0,26):
                            match=re.search(country[i],user_country)
                            if match:
                                country_id=user_country.replace(country[i],str(i))
                                print(country_id)
                                match = re.search(r'\d+', country_id)
                                country_id=match.group(0)
                                print(country_id)
                            else:
#                                 print("not found")
                                country_id='null'
                        # print("4"+user_country)
                        user_name_info = user_soup2.select('div.modules-membercenter-progress-header')[0].select('div.name')[0].text
                        # print("4"+user_name_info)
                        # user_point = user_soup2.select('div.points_info.tripcollectiveinfo')[0].text
                        # print("5"+user_point)
                        user_lv = user_soup2.select('div.level.tripcollectiveinfo')[0].text
                        print(user_lv)
                        match=re.search(r'\d',user_lv)
                        if match:
                            user_lv=match.group(0)
                            print(user_lv)
                        else:
                            user_lv='null'
                        try:        
                            num = len(user_soup2.select('li.content-info'))
                        except:
                            comm_times='null'
                            ranking_times='null'
                        for i in range(0,num):
                            comm = user_soup2.select('li.content-info')[i].select('a')[0]    
                        #     print(r['name'].decode('utf-8'))
                            try:
                                if comm['name'].decode('utf-8') == 'reviews': 
                                    print(comm.text)
                                    match=re.search(r'\d*',comm.text)
                                    if match:
                                        comm=match.group(0) 
                                        comm_times=comm   
                                        print(comm_times)
    #                                     print(type(comm_times))                                    
                            #         else:
                            #         comm_times='null'       
                                else:        
                                    comm_times='null'
                            except:
                                comm_times='null'
                            #====評論次數===#     
                            comm = user_soup2.select('li.content-info')[i].select('a')[0] 
                            try:
                                if comm['name'].decode('utf-8') == 'ratings':
                                    print(comm.text)
                                    match=re.search(r'\d*',comm.text)
                                    if match:            
                                        comm=match.group(0)                                    
                                        ranking_times=comm
                                        print(ranking_times)
#                                         print(type(ranking_times))
                            #         else:
                            #         comm_times='null'       
                                else:        
                                    ranking_times='null'
                            except:
                                ranking_times='null'        
                            #====評等次數===#       

                        # print("8"+user_cont_info)
                        # print(len(user_cont_info))
                        # match=re.search(r'\d+'+'評論',user_cont_info)
                        # user_cont_info=match.group(0)
                        # print(user_cont_info)                  
                    except:
                        user_target = 'null'
                        user_badge='null' 
                        user_lv='null' 
#                     finally:
#                         print("ok1")
        except IndexError:
                print("=====================")
                print("出現Skip")

In [1]:
gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8')                            
print(gender)
print("11111111")
s=len(user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8').split())
print(s)
#                             d=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8').split()[3]
#                             print(d)
for y in range(0,s):               
    gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8').split()[y]
    print(gender)                            
    s=['預設值','男性','女性','男','女']
    for i in range(0,5):
        match=re.search(s[i],gender)
        print("------------")                                
        if match:
            d=gender.replace(s[i],str(i))
            print("+++++++++")
            print(d)
#                                     gender=match.group(0)
#                                     print(gender)
#                                     gender=gender
#                                     print("***********")
#                                     print(gender)
#                                     print(type(gender))
        else:

            gender='null'
            print("not found")

NameError: name 'user_soup2' is not defined

In [ ]:
from __future__ import print_function
import sys
import requests as rs
from bs4 import BeautifulSoup as bs
from datetime import date,datetime
import json
# from splinter import Browser
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
home_url = 'https://www.tripadvisor.com.tw'
base_url='https://www.tripadvisor.com.tw/Attractions-g293913-Activities-oa{}-Taipei.html#ATTRACTION_LIST'
textNo=0
#景點分頁下載
for page in xrange(0,14,+1):
    title=(base_url.format(page*30))
#     print(title)
    res=rs.get(title)
    soup=bs(res.text,'html5lib')
    for n in soup.select('.photo_booking.non_generic .photo_link'):    
    #分頁網站連接下載
        try:
            url='https://www.tripadvisor.com.tw'+n['href']
            if(url.split('_')[1].split('-')[0]=='Review'):
                print(url)      
            else:
                url='null'
                print("怪怪的")
            #============分頁挑選========#    
            res=rs.get(url)
            soup=bs(res.text,'html5lib')        
            inner_url = url.replace('Reviews-','Reviews-or{}-')
#             print("1"+ inner_url)
            #==========各景點評論的初始頁==================#
            content = inner_url.split('-')[3].split('-')[0]
            total_page =int(soup.select('.unified.pagination .pageNum.taLnk')[-1]['data-page-number'])
            print(total_page)
            #=====各評論的總頁數===================#
            for page in xrange(0,total_page-1):
                inner_title = (inner_url.format(page*10))    
#                 print("2"+ inner_title)
                #===========各評論的評論內容========#    
                inner_page_res = rs.get(inner_title)
                inner_page_bs = bs(inner_page_res.text ,'html5lib')
                total_userId = len(inner_page_bs.select('div.memberBadging.g10n'))
                for num in range(0,total_userId,+1):               
                    try:
                        UID = inner_page_bs.select('div.memberBadging.g10n')[num].select('div')[0]['id']\
                            .split('-')[0]\
                            .replace('UID_','')
                            #使用者ID#
                        member_id = UID  
                        print(UID)                       
                        user_url = 'https://www.tripadvisor.com.tw/MemberOverlay?Mode=owa&uid={}&c=&src=&fus=false&partner=false&LsoId=&metaReferer=Attraction_Review'
                        user_infopage = (user_url.format(UID))
                        print(user_infopage)
                        print("++++++++++++++++++++++++++")
                        #各使用者簡介#
                        user_res = rs.get(user_infopage)
                        user_soup = bs(user_res.text,'html5lib')
#                                         print(user_soup.select('div.memberreviewbadge')[0].select('div.badgeinfo')[0].text)
                        def user_function (a,b):
                            return user_soup.select(a)[0].select(b)[0].text                
                        user_name = user_function('.memberOverlayRedesign.g10n','.username')                        
#                         print(user_name)                        
                        user_time = user_function('.memberOverlayRedesign.g10n','.memberdescription')
#                         print(type(user_time))
#                         print("1"+user_time)
                        user_counts = user_function('.memberOverlayRedesign.g10n','.counts')
                        member_remark=user_counts
#                         print("2"+member_remark)
                        user_homepage = home_url + user_soup.select('div.baseNav')[0].select('a')[0]['href']
                        print(user_homepage)
#                         user_homepage='https://www.tripadvisor.com.tw/members/doris7116'
                        # https://www.tripadvisor.com.tw/MemberProfile-a_uid.3806B087E1EEEDBA38F7E690310B5144
                        #     'https://www.tripadvisor.com.tw/members/doris7116'
                        user_res2 = rs.get(user_homepage)
                        print(user_res2)
                        user_soup2 = bs(user_res2.text,'html5lib')
                        user_year=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text
                        match=re.search(r'\d{2}-\d{2}', user_year)
                        if match:
                            print(user_year)
                            member_year=match.group(0)
                            print(member_year)
                        else:
                            print("not found")
                            member_year='null'
                        # print("3"+user_year)                        
                        try:
                            gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8')                            
                            print(gender)
#                             print("11111111")
                            s=len(user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8').split())
#                             print(s)                            
                            for y in range(0,s):               
                                gender=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.ageSince')[0].text.encode('utf-8').split()[y]
#                                 print(gender)                            
                                s=['預設值','男','女']
                                for i in range(0,3):
                                    match=re.search(s[i],gender)
#                                     print("------------")                                
                                    if match:
                                        d=gender.replace(s[i],str(i))
                                        match=re.search(r'\d', d)
                                        gender=match.group(0)
                                        print(gender)
                   
                        except:
                            gender='null'
                        #======gender==========#    
                        user_country=user_soup2.select('div.modules-membercenter-member-profile')[0].select('div.hometown')[0].text.encode('utf-8')
                        print(user_country)
                        country=['0','台灣','香港','馬來西亞','新加坡','美國','加拿大','澳洲','紐西蘭','英國','中國','韓國','日本','印尼','泰國','法國','德國','俄羅斯','哈薩克','葡萄牙','巴西','西班牙','阿根廷','秘魯','荷蘭','義大利','南韓']
                        for i in range(0,26):
                            match=re.search(country[i],user_country)
                            if match:
                                country_id=user_country.replace(country[i],str(i))
                                print(country_id)
                                match = re.search(r'\d+', country_id)
                                country_id=match.group(0)
                                print(country_id)
                            else:
                                country_id='null'
                        user_name_info = user_soup2.select('div.modules-membercenter-progress-header')[0].select('div.name')[0].text
                        user_lv = user_soup2.select('div.level.tripcollectiveinfo')[0].text
                        print(user_lv)
                        match=re.search(r'\d',user_lv)
                        if match:
                            user_lv=match.group(0)
                            print(user_lv)
                        else:
                            user_lv='null'
                        try:        
                            num = len(user_soup2.select('li.content-info'))
                        except:
                            comm_times='null'
                            ranking_times='null'
                        for i in range(0,num):
                            comm = user_soup2.select('li.content-info')[i].select('a')[0]                        
                            try:
                                if comm['name'].decode('utf-8') == 'reviews': 
                                    print(comm.text)
                                    match=re.search(r'\d*',comm.text)
                                    if match:
                                        comm=match.group(0) 
                                        comm_times=comm   
                                        print(comm_times) 
                                else:        
                                    comm_times='null'
                            except:
                                comm_times='null'
                            #====評論次數===#     
                            comm = user_soup2.select('li.content-info')[i].select('a')[0] 
                            try:
                                if comm['name'].decode('utf-8') == 'ratings':
                                    print(comm.text)
                                    match=re.search(r'\d*',comm.text)
                                    if match:            
                                        comm=match.group(0)                                    
                                        ranking_times=comm
                                        print(ranking_times)
#                                         print(type(ranking_times))
                            #         else:
                            #         comm_times='null'       
                                else:        
                                    ranking_times='null'
                            except:
                                ranking_times='null'        
                            #====評等次數===#       

                        # print("8"+user_cont_info)
                        # print(len(user_cont_info))
                        # match=re.search(r'\d+'+'評論',user_cont_info)
                        # user_cont_info=match.group(0)
                        # print(user_cont_info)                  
                    except:
                        user_target = 'null'
                        user_badge='null' 
                        user_lv='null' 
#                     finally:
#                         print("ok1")
        except IndexError:
                print("=====================")
                print("出現Skip")